In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import os
import requests
import re
from bs4 import BeautifulSoup 

In [2]:
df=pd.read_csv("results.csv", encoding='ISO-8859-1')
df=df.dropna()
df['Season'] = df['Season'].str[:4].astype('float')

# Web Scraping : Budget des équipes

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
base="https://www.transfermarkt.fr/premier-league/transfers/wettbewerb/GB1/plus/?saison_id="
urls=[]
for i in range(2000,2022):
    urls.append(base+str(i))
    
saison=1999

In [4]:
# création du df_budget
budget_data = {'season': [],'club': [],'budget': []}
df_budget = pd.DataFrame(budget_data)

In [5]:
for url in urls:
    page = requests.get(url,headers=headers)
    pageSoup = BeautifulSoup(page.content, 'html.parser')
    outer = pageSoup.findAll("div", {"class":"table-footer footer-border"})
    budget=[]
    for div in outer:
        if(len(div.findAll("span"))!=0):
            s = div.findAll("span")[0]
            budget.append(s)
    clubs = pageSoup.findAll("div",{"class":"table-header"})
    i=0
    saison=saison+1
    print("Saison : ",saison)
    for club,out in zip(clubs,budget) : 
        club=club.text.strip()
        out=out.text.strip()
        res=re.findall(r'-?\d+', out)
        if (len(res)==2):
            n=res[0]
            decimal = res[1]
            depense = float(n+'.'+decimal) 
        else:
            depense= float('0')
        i=i+1
        new_row = {'season':saison, 'club':club, 'budget':depense}
        #append row to the dataframe
        df_budget = pd.concat([df_budget,pd.DataFrame.from_records([new_row])])
        #print('\t',i,club,depense)

Saison :  2000
Saison :  2001
Saison :  2002
Saison :  2003
Saison :  2004
Saison :  2005
Saison :  2006
Saison :  2007
Saison :  2008
Saison :  2009
Saison :  2010
Saison :  2011
Saison :  2012
Saison :  2013
Saison :  2014
Saison :  2015
Saison :  2016
Saison :  2017
Saison :  2018
Saison :  2019
Saison :  2020
Saison :  2021


In [6]:
df_budget=df_budget.rename(columns={"season": "Season", "club": "HomeTeam"})

In [7]:
df_budget['HomeTeam'] = df_budget['HomeTeam'].replace({
    'FC Arsenal':'Arsenal', 
    'Chelsea FC':'Chelsea',
    'Leeds United':'Leeds',
    'Bradford City':'Bradford',
    'Leicester City':'Leicester',
    'Coventry City': 'Coventry',
    'Derby County':'Derby',
    'FC Liverpool':'Liverpool',
    'AFC Sunderland':'Sunderland',
    'Tottenham Hotspur':'Tottenham',
    'Manchester United':'Man United',
    'Ipswich Town':'Ipswich',
    'Middlesbrough FC':'Middlesbrough',
    'FC Everton':'Everton',
    'Manchester City':'Man City',
    'Newcastle United':'Newcastle',
    'FC Southampton':'Southampton',
    'West Ham United':'West Ham',
    'Bolton Wanderers':'Bolton',
    'Blackburn Rovers':'Blackburn',
    'FC Fulham':'Fulham',
    'Birmingham City':'Birmingham',
    'West Bromwich Albion':'West Brom',
    'FC Portsmouth':'Portsmouth',
    'Wolverhampton Wanderers':'Wolves',
    'Norwich City':'Norwich',
    'Crystal Palace':'Crystal Palace',
    'Wigan Athletic':'Wigan',
    'FC Reading':'Reading',
    'Sheffield United':'Sheffield United',
    'FC Watford':'Watford',
    'Hull City':'Hull',
    'Stoke City':'Stoke',
    'Burnley FC':'Burnley',
    'Blackpool FC':'Blackpool',
    'Queens Park Rangers':'QPR',
    'Swansea City':'Swansea',
    'Cardiff City':'Cardiff',
    'AFC Bournemouth':'Bournemouth',
    'Brighton & Hove Albion':'Brighton',
    'Huddersfield Town':'Huddersfield',
    'FC Brentford':'Brentford',
    'Charlton Athletic':'Charlton',})

In [8]:
df_concat = pd.merge(df, df_budget, how='left', left_on=['Season', 'HomeTeam'], right_on=['Season', 'HomeTeam'])
df_concat.sort_values(by=['DateTime'])
df_concat.rename(columns = {'budget':'BudgetHome'}, inplace = True)
df_budget=df_budget.rename(columns={"HomeTeam": "AwayTeam"})
df_concat = pd.merge(df_concat, df_budget, how='left', left_on=['Season', 'AwayTeam'], right_on=['Season', 'AwayTeam'])
df_concat.rename(columns = {'budget':'BudgetAway'}, inplace = True)
df_concat['DiffBudgets'] = df_concat['BudgetHome'] - df_concat['BudgetAway']

In [9]:
#df_concat.head(10)

In [10]:
df = df_concat
df['Diff But']= df['FTHG']-df['FTAG']
df['Journée']=0

In [11]:
list1 = [[i]*10 for i in range(1,39)]
flatten_list = list(np.concatenate(list1). flat)
for season in range(2000,2021):
    df.loc[df['Season'] == season, 'Journée'] = flatten_list
    
df['Journée'].value_counts().sort_values()

1     210
10    210
3     210
4     210
5     210
6     210
7     210
8     210
2     210
9     210
19    210
12    210
13    210
14    210
15    210
16    210
17    210
11    210
18    210
20    210
38    210
30    210
23    210
24    210
25    210
26    210
27    210
28    210
22    210
29    210
21    210
32    210
33    210
34    210
35    210
36    210
37    210
31    210
0     309
Name: Journée, dtype: int64

In [12]:
for season in range(2000,2021):
    print("Season ", season)
    print(len(df.loc[(df['Season'] == season) & ( (df['HomeTeam']=='Arsenal') | (df['AwayTeam']=='Arsenal')) , 'Journée'].unique()))


Season  2000
28
Season  2001
30
Season  2002
30
Season  2003
30
Season  2004
31
Season  2005
29
Season  2006
28
Season  2007
29
Season  2008
29
Season  2009
30
Season  2010
36
Season  2011
30
Season  2012
29
Season  2013
35
Season  2014
35
Season  2015
36
Season  2016
35
Season  2017
37
Season  2018
37
Season  2019
38
Season  2020
33


In [14]:
df.shape

(8289, 28)

In [15]:
df.to_csv(r'~/Downloads/all_season.csv')

# DATASET ENTIER

In [398]:
df = pd.read_csv('~/Downloads/all_season.csv')

In [399]:
df = df[(df.Season == 2008) ]
df.tail(10)

,Unnamed: 0,Season,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AF,HY,AY,HR,AR,BudgetHome,BudgetAway,DiffBudgets,Diff But,Journée
3410,3410,2008.0,2009-05-24T00:00:00Z,Arsenal,Stoke,4,1,H,4.0,1.0,...,11.0,1.0,0.0,0.0,0.0,-14.35,-33.19,18.84,3,38
3411,3411,2008.0,2009-05-24T00:00:00Z,Aston Villa,Newcastle,1,0,H,1.0,0.0,...,10.0,0.0,2.0,0.0,1.0,-48.38,5.93,-54.31,1,38
3412,3412,2008.0,2009-05-24T00:00:00Z,Blackburn,West Brom,0,0,D,0.0,0.0,...,11.0,0.0,1.0,1.0,0.0,14.82,-14.07,28.89,0,38
3413,3413,2008.0,2009-05-24T00:00:00Z,Fulham,Everton,0,2,A,0.0,1.0,...,10.0,0.0,0.0,0.0,0.0,-10.53,-11.78,1.25,-2,38
3414,3414,2008.0,2009-05-24T00:00:00Z,Hull,Man United,0,1,A,0.0,1.0,...,11.0,2.0,1.0,0.0,0.0,-17.95,-37.80,19.85,-1,38
3415,3415,2008.0,2009-05-24T00:00:00Z,Liverpool,Tottenham,3,1,H,1.0,0.0,...,4.0,0.0,1.0,0.0,0.0,-25.00,-55.90,30.90,2,38
3416,3416,2008.0,2009-05-24T00:00:00Z,Man City,Bolton,1,0,H,1.0,0.0,...,10.0,0.0,0.0,0.0,0.0,-130.85,-12.83,-118.02,1,38
3417,3417,2008.0,2009-05-24T00:00:00Z,Sunderland,Chelsea,2,3,A,0.0,0.0,...,7.0,1.0,1.0,0.0,0.0,-21.96,14.05,-36.01,-1,38
3418,3418,2008.0,2009-05-24T00:00:00Z,West Ham,Middlesbrough,2,1,H,1.0,0.0,...,10.0,0.0,1.0,0.0,0.0,52.83,-4.08,56.91,1,38
3419,3419,2008.0,2009-05-24T00:00:00Z,Wigan,Portsmouth,1,0,H,1.0,0.0,...,21.0,0.0,4.0,0.0,0.0,0.00,25.95,-25.95,1,38


In [400]:
for club in pd.unique(df['HomeTeam']):
    print("Club : ",club)
    print(len(df.loc[(df['Season'] == 2008) & ( (df['HomeTeam']==club) | (df['AwayTeam']==club)) , 'Journée'].unique()))

Club :  Arsenal
29
Club :  Bolton
33
Club :  Everton
36
Club :  Hull
36
Club :  Middlesbrough
37
Club :  Sunderland
35
Club :  West Ham
35
Club :  Aston Villa
32
Club :  Chelsea
31
Club :  Man United
31
Club :  Blackburn
29
Club :  Fulham
31
Club :  Liverpool
37
Club :  Newcastle
36
Club :  Stoke
36
Club :  Tottenham
36
Club :  West Brom
34
Club :  Man City
35
Club :  Wigan
34
Club :  Portsmouth
33


In [401]:
#df[(df['HomeTeam']=='Arsenal') | (df['AwayTeam']=='Arsenal')]

In [402]:
df.loc[((df['HomeTeam']=='Norwich') | (df['AwayTeam']=='Norwich'))&(df['Season']==2008)].shape

(0, 29)

In [403]:
"""df.loc[(df['HomeTeam']=='Brighton') & (df['AwayTeam']=='Chelsea') & (df['Season']==2019),'Journée']=21
df.loc[(df['HomeTeam']=='West Ham') & (df['AwayTeam']=='Liverpool')& (df['Season']==2019),'Journée']=18
df.loc[(df['HomeTeam']=='Liverpool') & (df['AwayTeam']=='Bournemouth')& (df['Season']==2019),'Journée']=29
df.loc[(df['HomeTeam']=='Arsenal') & (df['AwayTeam']=='West Ham')& (df['Season']==2019),'Journée']=29
df.loc[(df['HomeTeam']=='Sheffield United') & (df['AwayTeam']=='West Ham')& (df['Season']==2019),'Journée']=22
df.loc[(df['HomeTeam']=='Brighton') & (df['AwayTeam']=='Bournemouth')& (df['Season']==2019),'Journée']=20
df.loc[(df['HomeTeam']=='Watford') & (df['AwayTeam']=='Tottenham')& (df['Season']==2019),'Journée']=23
df.loc[(df['HomeTeam']=='Aston Villa') & (df['AwayTeam']=='Watford')& (df['Season']==2019),'Journée']=24
df.loc[(df['HomeTeam']=='Tottenham') & (df['AwayTeam']=='Brighton')& (df['Season']==2019),'Journée']=19
df.loc[(df['HomeTeam']=='Tottenham') & (df['AwayTeam']=='Watford')& (df['Season']==2019),'Journée']=9
df.loc[(df['HomeTeam']=='Man City') & (df['AwayTeam']=='Arsenal')& (df['Season']==2019),'Journée']=28
df.loc[(df['HomeTeam']=='Aston Villa') & (df['AwayTeam']=='Sheffield United')& (df['Season']==2019),'Journée']=28

df.loc[(df['HomeTeam']=='Everton') & (df['AwayTeam']=='Liverpool')& (df['Season']==2020),'Journée']=5
df.loc[(df['HomeTeam']=='Brighton') & (df['AwayTeam']=='Liverpool')& (df['Season']==2020),'Journée']=10
df.loc[(df['HomeTeam']=='Crystal Palace') & (df['AwayTeam']=='Liverpool')& (df['Season']==2020),'Journée']=14
df.loc[(df['HomeTeam']=='Leicester') & (df['AwayTeam']=='Liverpool')& (df['Season']==2020),'Journée']=24
df.loc[(df['HomeTeam']=='Man United') & (df['AwayTeam']=='Liverpool')& (df['Season']==2020),'Journée']=34

df.loc[(df['HomeTeam']=='West Ham') & (df['AwayTeam']=='Man City')& (df['Season']==2020),'Journée']=6
df.loc[(df['HomeTeam']=='Leeds') & (df['AwayTeam']=='West Ham')& (df['Season']==2020),'Journée']=12
df.loc[(df['HomeTeam']=='Everton') & (df['AwayTeam']=='West Ham')& (df['Season']==2020),'Journée']=17
df.loc[(df['HomeTeam']=='West Ham') & (df['AwayTeam']=='Burnley')& (df['Season']==2020),'Journée']=19
df.loc[(df['HomeTeam']=='Crystal Palace') & (df['AwayTeam']=='West Ham')& (df['Season']==2020),'Journée']=20
df.loc[(df['HomeTeam']=='Fulham') & (df['AwayTeam']=='West Ham')& (df['Season']==2020),'Journée']=23
df.loc[(df['HomeTeam']=='Man City') & (df['AwayTeam']=='West Ham')& (df['Season']==2020),'Journée']=26
df.loc[(df['HomeTeam']=='West Ham') & (df['AwayTeam']=='Leeds')& (df['Season']==2020),'Journée']=27
df.loc[(df['HomeTeam']=='Man United') & (df['AwayTeam']=='West Ham')& (df['Season']==2020),'Journée']=28"""


"df.loc[(df['HomeTeam']=='Brighton') & (df['AwayTeam']=='Chelsea') & (df['Season']==2019),'Journée']=21\ndf.loc[(df['HomeTeam']=='West Ham') & (df['AwayTeam']=='Liverpool')& (df['Season']==2019),'Journée']=18\ndf.loc[(df['HomeTeam']=='Liverpool') & (df['AwayTeam']=='Bournemouth')& (df['Season']==2019),'Journée']=29\ndf.loc[(df['HomeTeam']=='Arsenal') & (df['AwayTeam']=='West Ham')& (df['Season']==2019),'Journée']=29\ndf.loc[(df['HomeTeam']=='Sheffield United') & (df['AwayTeam']=='West Ham')& (df['Season']==2019),'Journée']=22\ndf.loc[(df['HomeTeam']=='Brighton') & (df['AwayTeam']=='Bournemouth')& (df['Season']==2019),'Journée']=20\ndf.loc[(df['HomeTeam']=='Watford') & (df['AwayTeam']=='Tottenham')& (df['Season']==2019),'Journée']=23\ndf.loc[(df['HomeTeam']=='Aston Villa') & (df['AwayTeam']=='Watford')& (df['Season']==2019),'Journée']=24\ndf.loc[(df['HomeTeam']=='Tottenham') & (df['AwayTeam']=='Brighton')& (df['Season']==2019),'Journée']=19\ndf.loc[(df['HomeTeam']=='Tottenham') & (df['A

In [404]:
for club in pd.unique(df['HomeTeam']):
    print("Club : ",club)
    print(len(df.loc[(df['Season'] == 2008) & ( (df['HomeTeam']==club) | (df['AwayTeam']==club)) , 'Journée'].unique()))

Club :  Arsenal
29
Club :  Bolton
33
Club :  Everton
36
Club :  Hull
36
Club :  Middlesbrough
37
Club :  Sunderland
35
Club :  West Ham
35
Club :  Aston Villa
32
Club :  Chelsea
31
Club :  Man United
31
Club :  Blackburn
29
Club :  Fulham
31
Club :  Liverpool
37
Club :  Newcastle
36
Club :  Stoke
36
Club :  Tottenham
36
Club :  West Brom
34
Club :  Man City
35
Club :  Wigan
34
Club :  Portsmouth
33


In [405]:
# CheckPoint

In [406]:
dict_club={
    'Arsenal FC':'Arsenal', 
    'Chelsea FC':'Chelsea',
    'Leeds United':'Leeds',
    'Bradford City':'Bradford',
    'Leicester City':'Leicester',
    'Coventry City': 'Coventry',
    'Derby County':'Derby',
    'Liverpool FC':'Liverpool',
    'AFC Sunderland':'Sunderland',
    'Tottenham Hotspur':'Tottenham',
    'Manchester United':'Man United',
    'Ipswich Town':'Ipswich',
    'Middlesbrough FC':'Middlesbrough',
    'Everton FC':'Everton',
    'Manchester City':'Man City',
    'Newcastle United':'Newcastle',
    'Southampton FC':'Southampton',
    'West Ham United':'West Ham',
    'Bolton Wanderers':'Bolton',
    'Blackburn Rovers':'Blackburn',
    'Fulham FC':'Fulham',
    'Birmingham City':'Birmingham',
    'West Bromwich Albion':'West Brom',
    'FC Portsmouth':'Portsmouth',
    'Wolverhampton Wanderers':'Wolves',
    'Norwich City':'Norwich',
    'Crystal Palace':'Crystal Palace',
    'Wigan Athletic':'Wigan',
    'FC Reading':'Reading',
    'Sheffield United':'Sheffield United',
    'Watford FC':'Watford',
    'Hull City':'Hull',
    'Stoke City':'Stoke',
    'Burnley FC':'Burnley',
    'Blackpool FC':'Blackpool',
    'Queens Park Rangers':'QPR',
    'Swansea City':'Swansea',
    'Cardiff City':'Cardiff',
    'AFC Bournemouth':'Bournemouth',
    'Brighton & Hove Albion':'Brighton',
    'Huddersfield Town':'Huddersfield',
    'FC Brentford':'Brentford',
    'Charlton Athletic':'Charlton',
    'Sunderland AFC' : 'Sunderland',
    'Reading FC' : 'Reading',
    'Portsmouth FC':'Portsmouth'
}
    
def change_name(club):
    if club in dict_club.keys() :
        return dict_club[club]
    return club

df['HnbPoints']=0
df['AnbPoints']=0

df['Hrank']=0
df['Arank']=0

In [407]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
base="https://www.mondefootball.fr/calendrier/eng-premier-league-2008-2009-spieltag/"
urls=[]
for i in range(1,39):
    urls.append(base+str(i)+'/')
    

In [408]:
j=0
for url in urls:
    j+=1
    rank=0
    page = requests.get(url,headers=headers)
    pageSoup = BeautifulSoup(page.content, 'html.parser')
    outer = pageSoup.findAll("table", {"class":"standard_tabelle"})[1]
    outer = outer.findAll("tr")
    points=0
    print("Journée ",j,"------------------------------------------")
    for t in outer:
        tds=t.findAll("td")
        i=0
        for td in tds:
            i+=1
            if(i==3):
                club=change_name(td.get_text(strip=True))
                print("\t Club : ",club,"classement : ",rank)
                if not (df.loc[(df['HomeTeam']==club) & (df['Journée']==j)]).empty :
                    df.loc[(df['HomeTeam']==club) & (df['Journée']==j),'Hrank']=rank
                if not (df.loc[(df['AwayTeam']==club) & (df['Journée']==j)]).empty :
                    df.loc[(df['AwayTeam']==club) & (df['Journée']==j),'Arank']=rank
            if(i==10):
                points=td.get_text(strip=True)
                print("\t nb de points : ",points)
            #add points to df 
            if not (df.loc[(df['HomeTeam']==club) & (df['Journée']==j)]).empty :
                df.loc[(df['HomeTeam']==club) & (df['Journée']==j),'HnbPoints']=points
            if not (df.loc[(df['AwayTeam']==club) & (df['Journée']==j)]).empty :
                df.loc[(df['AwayTeam']==club) & (df['Journée']==j),'AnbPoints']=points
        rank+=1


Journée  1 ------------------------------------------
	 Club :  Chelsea classement :  1
	 nb de points :  3
	 Club :  Aston Villa classement :  2
	 nb de points :  3
	 Club :  Bolton classement :  3
	 nb de points :  3
	 Club :  Blackburn classement :  4
	 nb de points :  3
	 Club :  Hull classement :  5
	 nb de points :  3
	 Club :  Middlesbrough classement :  6
	 nb de points :  3
	 Club :  West Ham classement :  7
	 nb de points :  3
	 Club :  Arsenal classement :  8
	 nb de points :  3
	 Club :  Liverpool classement :  9
	 nb de points :  3
	 Club :  Man United classement :  10
	 nb de points :  1
	 Club :  Newcastle classement :  11
	 nb de points :  1
	 Club :  Everton classement :  12
	 nb de points :  0
	 Club :  Fulham classement :  13
	 nb de points :  0
	 Club :  Tottenham classement :  14
	 nb de points :  0
	 Club :  Wigan classement :  15
	 nb de points :  0
	 Club :  Sunderland classement :  16
	 nb de points :  0
	 Club :  West Brom classement :  17
	 nb de points :  0


Journée  8 ------------------------------------------
	 Club :  Chelsea classement :  1
	 nb de points :  20
	 Club :  Liverpool classement :  2
	 nb de points :  20
	 Club :  Man United classement :  3
	 nb de points :  17
	 Club :  Hull classement :  4
	 nb de points :  17
	 Club :  Arsenal classement :  5
	 nb de points :  16
	 Club :  Aston Villa classement :  6
	 nb de points :  14
	 Club :  Portsmouth classement :  7
	 nb de points :  13
	 Club :  West Ham classement :  8
	 nb de points :  12
	 Club :  Blackburn classement :  9
	 nb de points :  11
	 Club :  Man City classement :  10
	 nb de points :  10
	 Club :  West Brom classement :  11
	 nb de points :  10
	 Club :  Sunderland classement :  12
	 nb de points :  9
	 Club :  Middlesbrough classement :  13
	 nb de points :  9
	 Club :  Wigan classement :  14
	 nb de points :  8
	 Club :  Bolton classement :  15
	 nb de points :  8
	 Club :  Everton classement :  16
	 nb de points :  8
	 Club :  Stoke classement :  17
	 nb de po

Journée  15 ------------------------------------------
	 Club :  Liverpool classement :  1
	 nb de points :  34
	 Club :  Chelsea classement :  2
	 nb de points :  33
	 Club :  Man United classement :  3
	 nb de points :  31
	 Club :  Arsenal classement :  4
	 nb de points :  26
	 Club :  Aston Villa classement :  5
	 nb de points :  25
	 Club :  Hull classement :  6
	 nb de points :  23
	 Club :  Everton classement :  7
	 nb de points :  22
	 Club :  Portsmouth classement :  8
	 nb de points :  22
	 Club :  Bolton classement :  9
	 nb de points :  20
	 Club :  Wigan classement :  10
	 nb de points :  19
	 Club :  Fulham classement :  11
	 nb de points :  19
	 Club :  Middlesbrough classement :  12
	 nb de points :  19
	 Club :  West Ham classement :  13
	 nb de points :  18
	 Club :  Stoke classement :  14
	 nb de points :  18
	 Club :  Man City classement :  15
	 nb de points :  17
	 Club :  Tottenham classement :  16
	 nb de points :  15
	 Club :  Newcastle classement :  17
	 nb de 

Journée  22 ------------------------------------------
	 Club :  Man United classement :  1
	 nb de points :  50
	 Club :  Liverpool classement :  2
	 nb de points :  47
	 Club :  Chelsea classement :  3
	 nb de points :  45
	 Club :  Aston Villa classement :  4
	 nb de points :  44
	 Club :  Arsenal classement :  5
	 nb de points :  41
	 Club :  Everton classement :  6
	 nb de points :  36
	 Club :  Wigan classement :  7
	 nb de points :  31
	 Club :  West Ham classement :  8
	 nb de points :  29
	 Club :  Portsmouth classement :  9
	 nb de points :  27
	 Club :  Hull classement :  10
	 nb de points :  27
	 Club :  Fulham classement :  11
	 nb de points :  26
	 Club :  Man City classement :  12
	 nb de points :  25
	 Club :  Blackburn classement :  13
	 nb de points :  24
	 Club :  Bolton classement :  14
	 nb de points :  23
	 Club :  Newcastle classement :  15
	 nb de points :  23
	 Club :  Sunderland classement :  16
	 nb de points :  23
	 Club :  Tottenham classement :  17
	 nb de

Journée  29 ------------------------------------------
	 Club :  Man United classement :  1
	 nb de points :  68
	 Club :  Chelsea classement :  2
	 nb de points :  61
	 Club :  Liverpool classement :  3
	 nb de points :  61
	 Club :  Arsenal classement :  4
	 nb de points :  52
	 Club :  Aston Villa classement :  5
	 nb de points :  52
	 Club :  Everton classement :  6
	 nb de points :  48
	 Club :  West Ham classement :  7
	 nb de points :  40
	 Club :  Wigan classement :  8
	 nb de points :  38
	 Club :  Fulham classement :  9
	 nb de points :  37
	 Club :  Man City classement :  10
	 nb de points :  35
	 Club :  Tottenham classement :  11
	 nb de points :  35
	 Club :  Bolton classement :  12
	 nb de points :  33
	 Club :  Hull classement :  13
	 nb de points :  33
	 Club :  Sunderland classement :  14
	 nb de points :  32
	 Club :  Blackburn classement :  15
	 nb de points :  30
	 Club :  Newcastle classement :  16
	 nb de points :  29
	 Club :  Portsmouth classement :  17
	 nb de

Journée  36 ------------------------------------------
	 Club :  Man United classement :  1
	 nb de points :  86
	 Club :  Liverpool classement :  2
	 nb de points :  80
	 Club :  Chelsea classement :  3
	 nb de points :  77
	 Club :  Arsenal classement :  4
	 nb de points :  68
	 Club :  Aston Villa classement :  5
	 nb de points :  58
	 Club :  Everton classement :  6
	 nb de points :  57
	 Club :  Fulham classement :  7
	 nb de points :  50
	 Club :  Tottenham classement :  8
	 nb de points :  48
	 Club :  West Ham classement :  9
	 nb de points :  48
	 Club :  Man City classement :  10
	 nb de points :  47
	 Club :  Wigan classement :  11
	 nb de points :  42
	 Club :  Stoke classement :  12
	 nb de points :  42
	 Club :  Bolton classement :  13
	 nb de points :  40
	 Club :  Blackburn classement :  14
	 nb de points :  40
	 Club :  Portsmouth classement :  15
	 nb de points :  38
	 Club :  Sunderland classement :  16
	 nb de points :  36
	 Club :  Newcastle classement :  17
	 nb d

In [409]:
df.loc[(df['Arank']>=21) | (df['Hrank']>=21) |(df['Arank']<1) |(df['Arank']<1)]

,Unnamed: 0,Season,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AR,BudgetHome,BudgetAway,DiffBudgets,Diff But,Journée,HnbPoints,AnbPoints,Hrank,Arank


In [410]:
#df.to_csv(r'~/Downloads/Yaniv2017.csv')

In [411]:
df

,Unnamed: 0,Season,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AR,BudgetHome,BudgetAway,DiffBudgets,Diff But,Journée,HnbPoints,AnbPoints,Hrank,Arank
3040,3040,2008.0,2008-08-16T00:00:00Z,Arsenal,West Brom,1,0,H,1.0,0.0,...,0.0,-14.35,-14.07,-0.28,1,1,3,0,8,17
3041,3041,2008.0,2008-08-16T00:00:00Z,Bolton,Stoke,3,1,H,3.0,0.0,...,0.0,-12.83,-33.19,20.36,2,1,3,0,3,19
3042,3042,2008.0,2008-08-16T00:00:00Z,Everton,Blackburn,2,3,A,1.0,1.0,...,0.0,-11.78,14.82,-26.60,-1,1,0,3,12,4
3043,3043,2008.0,2008-08-16T00:00:00Z,Hull,Fulham,2,1,H,1.0,1.0,...,0.0,-17.95,-10.53,-7.42,1,1,3,0,5,13
3044,3044,2008.0,2008-08-16T00:00:00Z,Middlesbrough,Tottenham,2,1,H,0.0,0.0,...,0.0,-4.08,-55.90,51.82,1,1,3,0,6,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,3415,2008.0,2009-05-24T00:00:00Z,Liverpool,Tottenham,3,1,H,1.0,0.0,...,0.0,-25.00,-55.90,30.90,2,38,86,51,2,8
3416,3416,2008.0,2009-05-24T00:00:00Z,Man City,Bolton,1,0,H,1.0,0.0,...,0.0,-130.85,-12.83,-118.02,1,38,50,41,10,13
3417,3417,2008.0,2009-05-24T00:00:00Z,Sunderland,Chelsea,2,3,A,0.0,0.0,...,0.0,-21.96,14.05,-36.01,-1,38,36,83,16,3
3418,3418,2008.0,2009-05-24T00:00:00Z,West Ham,Middlesbrough,2,1,H,1.0,0.0,...,0.0,52.83,-4.08,56.91,1,38,51,32,9,19


In [412]:
df.insert(0, 'Match_id', range(0, len(df)))

In [413]:
df

,Match_id,Unnamed: 0,Season,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AR,BudgetHome,BudgetAway,DiffBudgets,Diff But,Journée,HnbPoints,AnbPoints,Hrank,Arank
3040,0,3040,2008.0,2008-08-16T00:00:00Z,Arsenal,West Brom,1,0,H,1.0,...,0.0,-14.35,-14.07,-0.28,1,1,3,0,8,17
3041,1,3041,2008.0,2008-08-16T00:00:00Z,Bolton,Stoke,3,1,H,3.0,...,0.0,-12.83,-33.19,20.36,2,1,3,0,3,19
3042,2,3042,2008.0,2008-08-16T00:00:00Z,Everton,Blackburn,2,3,A,1.0,...,0.0,-11.78,14.82,-26.60,-1,1,0,3,12,4
3043,3,3043,2008.0,2008-08-16T00:00:00Z,Hull,Fulham,2,1,H,1.0,...,0.0,-17.95,-10.53,-7.42,1,1,3,0,5,13
3044,4,3044,2008.0,2008-08-16T00:00:00Z,Middlesbrough,Tottenham,2,1,H,0.0,...,0.0,-4.08,-55.90,51.82,1,1,3,0,6,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,375,3415,2008.0,2009-05-24T00:00:00Z,Liverpool,Tottenham,3,1,H,1.0,...,0.0,-25.00,-55.90,30.90,2,38,86,51,2,8
3416,376,3416,2008.0,2009-05-24T00:00:00Z,Man City,Bolton,1,0,H,1.0,...,0.0,-130.85,-12.83,-118.02,1,38,50,41,10,13
3417,377,3417,2008.0,2009-05-24T00:00:00Z,Sunderland,Chelsea,2,3,A,0.0,...,0.0,-21.96,14.05,-36.01,-1,38,36,83,16,3
3418,378,3418,2008.0,2009-05-24T00:00:00Z,West Ham,Middlesbrough,2,1,H,1.0,...,0.0,52.83,-4.08,56.91,1,38,51,32,9,19


In [414]:
teams = df.HomeTeam.unique()
teams

array(['Arsenal', 'Bolton', 'Everton', 'Hull', 'Middlesbrough',
       'Sunderland', 'West Ham', 'Aston Villa', 'Chelsea', 'Man United',
       'Blackburn', 'Fulham', 'Liverpool', 'Newcastle', 'Stoke',
       'Tottenham', 'West Brom', 'Man City', 'Wigan', 'Portsmouth'],
      dtype=object)

In [415]:
def homeAway(df, team, shift):
    df['H-'+str(shift)]=0
    df['A-'+str(shift)]=0

    
    for i in range(shift, 38):
        if df['HomeTeam'].iloc[i] == team:
            if df['HomeTeam'].iloc[i-shift] == team: 
                df['H-'+str(shift)].iloc[i] = df['Nouvelle'+str(shift)].iloc[i]
            else:
                df['H-'+str(shift)].iloc[i] = -df['Nouvelle'+str(shift)].iloc[i]
        else :
            if df['AwayTeam'].iloc[i] == team:
                if df['HomeTeam'].iloc[i-shift] == team: 
                    df['A-'+str(shift)].iloc[i] = df['Nouvelle'+str(shift)].iloc[i]
                else:
                    df['A-'+str(shift)].iloc[i] = -df['Nouvelle'+str(shift)].iloc[i]
    return df

def creation_forme(df, Teams, shift):
    res = {'Match_id': [], 'Season': [], 'DateTime': [], 'HomeTeam': [], 'AwayTeam': [], 'FTHG': [],
       'FTAG': [], 'FTR': [], 'HTHG': [], 'HTAG': [], 'HTR': [], 'Referee': [], 'HS': [], 'AS': [], 'HST': [],
       'AST': [], 'HC': [], 'AC': [], 'HF': [], 'AF': [], 'HY': [], 'AY': [], 'HR': [], 'AR': [], 'BudgetHome': [],
       'BudgetAway': [], 'DiffBudgets': [], 'Diff But': [], 'HNA': [], 'Journée': []}
    df_res = pd.DataFrame(res)
    for team in Teams:
        df_Team = df[(df['HomeTeam']==team) | (df['AwayTeam']==team)]
        df_Team['Nouvelle'+str(shift)] = df_Team['Diff But'].shift(shift)
        df_Team = homeAway(df_Team, team, shift)
        df_res = pd.concat([df_Team, df_res])
    
    df_res = df_res.sort_values('Match_id')
    df_res = df_res.groupby(['Match_id'])[('A-'+str(shift), 'H-'+str(shift))].sum()
    df_res.iloc[0:10*shift] = -999
    df_res.reset_index(inplace=True)
    return df_res


df_H1 = creation_forme(df, teams, 1)
df_H2 = creation_forme(df, teams, 2)
df_H3 = creation_forme(df, teams, 3)

/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Team['Nouvelle'+str(shift)] = df_Team['Diff But'].shift(shift)
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['H-'+str(shift)]=0
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Team['Nouvelle'+str(shift)] = df_Team['Diff But'].shift(shift)
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['H-'+str(shift)]=0
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Team['Nouvelle'+str(shift)] = df_Team['Diff But'].shift(shift)
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['H-'+str(shift)]=0
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Team['Nouvelle'+str(shift)] = df_Team['Diff But'].shift(shift)
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['H-'+str(shift)]=0
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Team['Nouvelle'+str(shift)] = df_Team['Diff But'].shift(shift)
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['H-'+str(shift)]=0
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

...

/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Team['Nouvelle'+str(shift)] = df_Team['Diff But'].shift(shift)
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['H-'+str(shift)]=0
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Team['Nouvelle'+str(shift)] = df_Team['Diff But'].shift(shift)
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['H-'+str(shift)]=0
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Team['Nouvelle'+str(shift)] = df_Team['Diff But'].shift(shift)
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['H-'+str(shift)]=0
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Team['Nouvelle'+str(shift)] = df_Team['Diff But'].shift(shift)
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['H-'+str(shift)]=0
/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/xj/c0329w812gv7rs0jcpvskjnc0000gn/T/ipykernel_68331/3837708634.py:33: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_res = df_res.groupby(['Match_id'])[('A-'+str(shift), 'H-'+str(shift))].sum()


In [416]:
df3 = pd.merge(df_H3, df_H2, how='inner', on='Match_id')
df_final = pd.merge(df3, df_H1)
df_final

,Match_id,A-3,H-3,A-2,H-2,A-1,H-1
0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,2.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,3.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,4.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
...,...,...,...,...,...,...,...
375,375.0,1.0,3.0,0.0,3.0,1.0,2.0
376,376.0,0.0,2.0,0.0,-2.0,0.0,-1.0
377,377.0,2.0,-2.0,3.0,0.0,2.0,-2.0
378,378.0,-2.0,1.0,-2.0,-3.0,0.0,-2.0


In [417]:
#df_final.to_csv(r'~/Downloads/Samuel2017.csv')

In [418]:
data = pd.merge(df_final,df, how='inner', on='Match_id')

In [419]:
data

,Match_id,A-3,H-3,A-2,H-2,A-1,H-1,Unnamed: 0,Season,DateTime,...,AR,BudgetHome,BudgetAway,DiffBudgets,Diff But,Journée,HnbPoints,AnbPoints,Hrank,Arank
0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3040,2008.0,2008-08-16T00:00:00Z,...,0.0,-14.35,-14.07,-0.28,1,1,3,0,8,17
1,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3041,2008.0,2008-08-16T00:00:00Z,...,0.0,-12.83,-33.19,20.36,2,1,3,0,3,19
2,2.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3042,2008.0,2008-08-16T00:00:00Z,...,0.0,-11.78,14.82,-26.60,-1,1,0,3,12,4
3,3.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3043,2008.0,2008-08-16T00:00:00Z,...,0.0,-17.95,-10.53,-7.42,1,1,3,0,5,13
4,4.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3044,2008.0,2008-08-16T00:00:00Z,...,0.0,-4.08,-55.90,51.82,1,1,3,0,6,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375.0,1.0,3.0,0.0,3.0,1.0,2.0,3415,2008.0,2009-05-24T00:00:00Z,...,0.0,-25.00,-55.90,30.90,2,38,86,51,2,8
376,376.0,0.0,2.0,0.0,-2.0,0.0,-1.0,3416,2008.0,2009-05-24T00:00:00Z,...,0.0,-130.85,-12.83,-118.02,1,38,50,41,10,13
377,377.0,2.0,-2.0,3.0,0.0,2.0,-2.0,3417,2008.0,2009-05-24T00:00:00Z,...,0.0,-21.96,14.05,-36.01,-1,38,36,83,16,3
378,378.0,-2.0,1.0,-2.0,-3.0,0.0,-2.0,3418,2008.0,2009-05-24T00:00:00Z,...,0.0,52.83,-4.08,56.91,1,38,51,32,9,19


In [422]:
data.info()
data.columns
data = data.rename(columns={'Unnamed: 0': 'TotalMatch_id'})

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 379
Data columns (total 40 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Match_id       380 non-null    float64
 1   A-3            380 non-null    float64
 2   H-3            380 non-null    float64
 3   A-2            380 non-null    float64
 4   H-2            380 non-null    float64
 5   A-1            380 non-null    float64
 6   H-1            380 non-null    float64
 7   TotalMatch_id  380 non-null    int64  
 8   Season         380 non-null    float64
 9   DateTime       380 non-null    object 
 10  HomeTeam       380 non-null    object 
 11  AwayTeam       380 non-null    object 
 12  FTHG           380 non-null    int64  
 13  FTAG           380 non-null    int64  
 14  FTR            380 non-null    object 
 15  HTHG           380 non-null    float64
 16  HTAG           380 non-null    float64
 17  HTR            380 non-null    object 
 18  Referee   

In [423]:
data

,Match_id,A-3,H-3,A-2,H-2,A-1,H-1,TotalMatch_id,Season,DateTime,...,AR,BudgetHome,BudgetAway,DiffBudgets,Diff But,Journée,HnbPoints,AnbPoints,Hrank,Arank
0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3040,2008.0,2008-08-16T00:00:00Z,...,0.0,-14.35,-14.07,-0.28,1,1,3,0,8,17
1,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3041,2008.0,2008-08-16T00:00:00Z,...,0.0,-12.83,-33.19,20.36,2,1,3,0,3,19
2,2.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3042,2008.0,2008-08-16T00:00:00Z,...,0.0,-11.78,14.82,-26.60,-1,1,0,3,12,4
3,3.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3043,2008.0,2008-08-16T00:00:00Z,...,0.0,-17.95,-10.53,-7.42,1,1,3,0,5,13
4,4.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3044,2008.0,2008-08-16T00:00:00Z,...,0.0,-4.08,-55.90,51.82,1,1,3,0,6,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375.0,1.0,3.0,0.0,3.0,1.0,2.0,3415,2008.0,2009-05-24T00:00:00Z,...,0.0,-25.00,-55.90,30.90,2,38,86,51,2,8
376,376.0,0.0,2.0,0.0,-2.0,0.0,-1.0,3416,2008.0,2009-05-24T00:00:00Z,...,0.0,-130.85,-12.83,-118.02,1,38,50,41,10,13
377,377.0,2.0,-2.0,3.0,0.0,2.0,-2.0,3417,2008.0,2009-05-24T00:00:00Z,...,0.0,-21.96,14.05,-36.01,-1,38,36,83,16,3
378,378.0,-2.0,1.0,-2.0,-3.0,0.0,-2.0,3418,2008.0,2009-05-24T00:00:00Z,...,0.0,52.83,-4.08,56.91,1,38,51,32,9,19


In [424]:
data.to_csv(r'~/Downloads/2008.csv')